## Analysis of double positive granules Using Phasor

This section describes the function `analize_coloc_with_phasor`, which leverages the phasor analysis approach using a plugin from napari napari_flim_phasor_plotter.
The function specifically targets colocalizing granules, segmenting intensity images of two fluorophores to evaluate average lifetimes of them.

### Function Overview

The function fits the FLIM data using the phasor approach and performs the following steps:

1. **Loading Data**: Reads FLIM data from the provided file paths.
2. **Processing Channels**: Selects the appropriate channel from the data for analysis.
3. **Phasor Analysis**: Computes the phasor components (G and S) and the corresponding lifetime (tau) in picoseconds.
4. **Intensity Segmentation**: Segments intensity images to identify double positive objects.
5. **Feature Extraction**: Measures features from the segmented objects and generates a dataframe with these features.
6. **Image Generation**: Creates a multichannel image containing intensity, G, S, and phasor lifetime information.



## Add a package to pathon path 

In [7]:
import sys
import os

# Manually set the parent directory path
notebook_dir = os.getcwd()  # This gets the current working directory of the notebook
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the parent directory to the system path
sys.path.insert(0, parent_dir)

## Import an example function from package 

In [8]:
from flim_processing import analize_coloc_with_phasor

## Provide the path to raw data in PTU format 
since here the example data is provided, the path is relative. This should be modified according to the needs 

In [9]:
# path to ptu files
data_path_1 = r"../data/Image_phasor_example/cl1-y1_seq1.ptu"
data_path_2 = r"../data/Image_phasor_example/cl1-y1_seq2.ptu"


## Example of function use 

In [10]:
df,multichannel_image=analize_coloc_with_phasor(data_path_1,data_path_2)
df

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!



,label,area,average_fluor_intensity,average_G,average_S,average_phasor_tau_ps
0,1,30.0,45.033333,0.490562,0.369047,1590.209798
1,2,67.0,324.641791,0.539336,0.419453,1556.615017
2,3,47.0,376.914894,0.551417,0.417819,1512.021149
3,4,34.0,296.647059,0.513341,0.422035,1645.194002
4,5,41.0,427.975610,0.535211,0.423837,1584.184398
5,6,35.0,274.314286,0.515787,0.422564,1641.075226
6,7,55.0,419.854545,0.554454,0.416565,1499.615048
7,8,91.0,180.846154,0.473515,0.420401,1785.164999
8,9,106.0,415.716981,0.539981,0.422094,1560.470731
9,10,80.0,201.787500,0.518012,0.421503,1637.241005


## Representation of the output image
This image contains intensity, G, S, and phasor lifetime information, as well as result of the segmentation in the form of parametric map with average lifetime value per object

In [ ]:
import matplotlib.pyplot as plt
    
fig, axes = plt.subplots(1, 3, squeeze=False, figsize=(10, 5))
#summed intensity f CFP (input image)
axes[0, 0].imshow(report_multichannel_image[:, :, 0])
axes[0, 0].axis("off")
#second chanell summed intensity (input image)
axes[0, 1].imshow(report_multichannel_image[:, :, 5])
axes[0, 1].axis("off")
#result of the segmentation (param map)
axes[0, 2].imshow(report_multichannel_image[:, :, 4])
axes[0, 2].axis("off")

fig.suptitle(str('example_of_output_image'))
plt.tight_layout()
#fig.savefig(output_folder_path / (image_id + '_plots.png'))
#plt.close()


## Example of the Phasor plotting
Uses G and S coordinates from the output table

In [ ]:

import seaborn as sns
frequency=80 
laser_frequency=80

def add_phasor_circle(ax):
    '''
    Generate FLIM universal semi-circle plot
    '''
    import numpy as np
    angles = np.linspace(0, np.pi, 180)
    x = (np.cos(angles) + 1) / 2
    y = np.sin(angles) / 2
    ax.plot(x, y, 'red', alpha=0.3)
    return ax

def add_tau_line(ax, tau, frequency):
    '''
    Add tau lines to the phasor plot
    '''
    import numpy as np
    frequency = frequency * 1E6  # MHz to Hz
    w = 2 * np.pi * frequency  # Hz to radians/s
    tau = tau * 1E-9  # nanoseconds to seconds
    g = 1 / (1 + ((w * tau)**2))
    s = (w * tau) / (1 + ((w * tau)**2))
    dot, = ax.plot(g, s, marker='o', mfc='none')
    array = np.linspace(0, g, 50)
    y = (array * s / g)
    #ax.plot(array, y, color=dot.get_color())
    return g, s

fig, ax = plt.subplots()
sns.scatterplot(data=df, x = 'average_G', y = 'average_S', ax = ax,zorder=0.002,alpha=0.2)
add_phasor_circle(ax)

tau_list = [0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for tau in tau_list:
    g, s = add_tau_line(ax, tau, laser_frequency)
    text = str(tau) + ' ns'
    plt.text(g, s, text)